In [ ]:
!pip3 install requests
!pip3 install beautifulsoup4
!pip3 install selenium
!pip3 install webdriver-manager
!apt install chromium-chromedriver
!pip3 install modin[all] 
!pip3 install summa
!pip3 install git+https://github.com/LIAAD/yake

In [ ]:
import modin.pandas as pd
import re
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
from IPython.core.display import clear_output
from random import randint
from requests import get
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
from time import time
start_time = time()
from warnings import warn
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from os import path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer 
from scipy.sparse import coo_matrix
from summa import keywords
from yake import KeywordExtractor

###DATA MINING: JOB SCRAPING

In [ ]:
#replace variables here.

#driver.get("https://www.linkedin.com/uas/login")
#sleep(3)
#email=driver.find_element_by_id("username")
#email.send_keys('self.username')
#password=driver.find_element_by_id("password")
#password.send_keys('self.password')
#sleep(3)
#password.send_keys(Keys.RETURN)

position = 'Software Developer'
location = 'Silicon Valley'

positionL = position.replace(" ","%20")
locationL = location.replace(" ","%20")

url = "https://www.linkedin.com/jobs/search/?keywords={}&location={}&sortBy=DD".format(positionL, locationL)

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
driver = webdriver.Chrome('chromedriver',options=chrome_options)

driver.get(url)
sleep(3)
action = ActionChains(driver)

In [ ]:
#Scrape more than the initial 25 jobs, scrolling the page
sleep(1)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(1)
driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
sleep(1)

In [ ]:
# parsing the visible webpage
pageSource = driver.page_source
lxml_soup = BeautifulSoup(pageSource, 'lxml')

# searching for all job containers
job_container = lxml_soup.find('ul', class_ = 'jobs-search__results-list')

print('You are scraping information about {} jobs.'.format(len(job_container)))

In [ ]:
# setting up list for job information
job_id = []
job_desc = []

# for loop for job title, company, id, location and date posted
for job in job_container:
    
    # linkedin job id
    job_ids = job.find('a', href=True)['href']
    job_ids = re.findall(r'(?!-)([0-9]*)(?=\?)',job_ids)[0]
    job_id.append(job_ids)

# for loop for job description and criterias
for x in range(1,len(job_id)+1):
        
    try:
     
    # clicking on different job containers to view information about the job
       job_xpath = '/html/body/main/div/section/ul/li[{}]/img'.format(x)
       driver.find_element_by_xpath(job_xpath).click()
       sleep(5)
        
       # job description
       jobdescBut = '/html/body/main/section/div[2]/section[2]/div/section/button[1]'
       driver.find_element_by_xpath(jobdescBut).click()
       sleep(5)
       jobdesc_xpath = '/html/body/main/section/div[2]/section[2]/div/section/div'
       job_descs = driver.find_element_by_xpath(jobdesc_xpath).text
       job_desc.append(job_descs)
     
     except Exception:
       pass
     x = x+1

driver.close()

In [ ]:
print(len(job_desc))

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)
driver = webdriver.Chrome('chromedriver',options=chrome_options)

driver.get('https://indeed.com')
sleep(2)

initial_search_button = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[3]/div[1]/div/div/div/form/div[3]/button')
initial_search_button.click()

#close_popup = driver.find_element_by_id("popover-close-link")
#close_popup.click()

advanced_search = driver.find_element_by_xpath("//a[contains(text(),'Advanced Job Search')]")
advanced_search.click()

search_job = driver.find_element_by_xpath('//input[@id="as_and"]')
search_job.send_keys([position])
#set display limit of 50 results per page
display_limit = driver.find_element_by_xpath('//select[@id="limit"]//option[@value="50"]')
display_limit.click()
#sort by date
sort_option = driver.find_element_by_xpath('//select[@id="sort"]//option[@value="date"]')
sort_option.click()

search_button = driver.find_element_by_xpath('//*[@id="fj"]')
search_button.click()

In [ ]:
#let the driver wait 3 seconds to locate the element before exiting out
driver.implicitly_wait(3) 

links =[]
descriptions=[]

job_card = driver.find_elements_by_xpath('//div[contains(@class,"clickcard")]')

for job in job_card:
    links.append(job.find_element_by_xpath('.//h2[@class="title"]//a').get_attribute(name="href"))

In [ ]:
descriptions=[]
for link in links:
    driver.get(link)
    sleep(2)
    jd = driver.find_element_by_xpath('//div[@id="jobDescriptionText"]').text
    descriptions.append(jd)

In [ ]:
print(len(job_desc))
print(position)
print(len(descriptions))
job_desc.extend(descriptions)
print(len(job_desc))

In [ ]:
# creating a dataframe
job_data = pd.DataFrame({
'Description':job_desc,
})

# cleaning description column
job_data['Description'] = job_data['Description'].str.replace('\n',' ')

print(job_data.info())

In [ ]:
job_data.to_csv(position+'JobDesc.csv', index=0)

###NLP PROCESSING

In [ ]:
dataset = pd.read_csv('./'+position+'JobDesc.csv', delimiter = ',')

stop_words = set(stopwords.words("english"))
stop_words.update(stopwords.words("italian"))

# Load a set of custom stop words from a text file (one stopword per line)
csw = set(line.strip() for line in open('./custom-stopwords.txt'))
csw = [sw.lower() for sw in csw]
stop_words.update(csw)

print(len(stop_words))

In [ ]:
corpus = []
dataset['word_count'] = dataset['Description'].apply(lambda x: len(str(x).split(" ")))
ds_count = len(dataset.word_count)

for i in range(0, ds_count):
    # Remove punctuation
    text = re.sub('[^a-zA-Z]', ' ', str(dataset['Description'][i]))
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    
    # Remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    
    # Convert to list from string
    text = text.split()
    
    # Stemming
    ps=PorterStemmer()
    
    # Lemmatization
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in stop_words] 
    text = " ".join(text)
    corpus.append(text)

###GRAPHS


In [ ]:
%matplotlib inline
wordcloud = WordCloud(
                          background_color='white',
                          width=1600, 
                          height=800,
                          stopwords=stop_words,
                          max_words=150,
                         ).generate(str(corpus))

fig = plt.figure(figsize=(20,10))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
fig.savefig(position + " WordCloud.png")

In [ ]:
# View most frequently occuring keywords
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

# Convert most freq words to dataframe for plotting bar plot, save as CSV
top_words = get_top_n_words(corpus, n = 20)
top_df = pd.DataFrame(top_words)
top_df.columns=["Keyword", "Frequency"]
top_df.to_csv(position + '_top_words.csv')

# Barplot of most freq words
sns.set(rc={'figure.figsize':(13,8)})
g = sns.barplot(x=top_df['Keyword'].tolist(), y=top_df['Frequency'].tolist())
g.set_xticklabels(g.get_xticklabels(), rotation=45)
g.figure.savefig(position + "_keyword.png", bbox_inches = "tight")

In [ ]:
def get_top_n2_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range = (2,2), max_features = 2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec1.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]

# Convert most freq bigrams to dataframe for plotting bar plot, save as CSV
top2_words = get_top_n2_words(corpus, n = 20)
top2_df = pd.DataFrame(top2_words)
top2_df.columns=["Bi-gram", "Frequency"]
top2_df.to_csv(position + '_bigrams.csv')

# Barplot of most freq Bi-grams
sns.set(rc={'figure.figsize':(13,8)})
h=sns.barplot(x=top2_df['Bi-gram'].tolist(), y=top2_df['Frequency'].tolist())
h.set_xticklabels(h.get_xticklabels(), rotation=75)
h.figure.savefig(position + "_bi-gram.png", bbox_inches = "tight")

In [ ]:
def get_top_n3_words(corpus, n=None):
    vec1 = CountVectorizer(ngram_range = (3,3), max_features=2000).fit(corpus)
    bag_of_words = vec1.transform(corpus)
    sum_words = bag_of_words.sum(axis = 0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec1.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq[:n]

# Convert most freq trigrams to dataframe for plotting bar plot, save as CSV
top3_words = get_top_n3_words(corpus, n = 20)
top3_df = pd.DataFrame(top3_words)
top3_df.columns=["Tri-gram", "Frequency"]
top3_df.to_csv(position + '_trigrams.csv')

# Barplot of most freq Tri-grams
sns.set(rc={'figure.figsize':(13,8)})
j=sns.barplot(x=top3_df['Tri-gram'].tolist(), y=top3_df['Frequency'].tolist())
j.set_xticklabels(j.get_xticklabels(), rotation=75)
j.figure.savefig(position + "_tri-gram.png", bbox_inches = "tight")

###TF-IDF

In [ ]:
cv = CountVectorizer(max_df=0.8,stop_words=stop_words, max_features=10000, ngram_range=(1,3))
X = cv.fit_transform(corpus)
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(X)

# Get feature names
feature_names = cv.get_feature_names()
 
# Fetch document for which keywords needs to be extracted
doc=str(corpus[:])
 
# Generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))

def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=25):
    
    # Use only topn items from vector
    sorted_items = sorted_items[:topn]
    score_vals = []
    feature_vals = []
    
    # Word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        # Keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    # Create tuples of feature,score
    # Results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    return results

# Sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())

# Extract only the top n
keywords=extract_topn_from_vector(feature_names,sorted_items,25)
 
print("Keywords:")
for k in keywords:
    print(k,keywords[k])

#import csv
#with open(file_prefix + 'td_idf.csv', 'w', newline="") as csv_file:  
#    writer = csv.writer(csv_file)
#    writer.writerow(["Keyword", "Importance"])
#    for key, value in keywords.items():
#       writer.writerow([key, value])

###KEYWORD EXTRACTOR WITH NEURAL NETWORKS

In [ ]:
from summa import keywords
print("Keywords from summa: ", keywords.keywords(str(corpus), words=10).replace("\n"," "))

from yake import KeywordExtractor
kw_extractor = KeywordExtractor(lan="en", n=1, top=10)
keywords = kw_extractor.extract_keywords(text=str(corpus))
keywords = [x for x, y in keywords]
print("Keywords with YAKE: ", keywords)